In [7]:
!pip install boto3
!pip install python-dotenv
!pip install requests
!pip install progress

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9645 sha256=931dd01c8deafb760b3a4ccb8e27b14b050ac5862dac00940d9da8e08d763f39
  Stored in directory: c:\users\cyber\appdata\local\pip\cache\wheels\b5\b9\86\f1bcc2a1de592673c4192d9459c0da1100d70212f38b6bd2a4
Successfully built progress


In [2]:
import os
import re
import boto3
import requests
from dotenv import load_dotenv

load_dotenv()
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.environ.get('REGION_NAME')

dynamodb = boto3.resource('dynamodb', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, region_name=REGION_NAME)

access_token = 'BQCdtSN7Ns_GILZosyUqPUAnjuc2eQ4ErvY9BTxPhegunxr7ZEohPrFlC05G43AWaPLtDkP1gDDwDfEsnADkJMYjXUzJgJ94RAG3MAxDBiLNHIl2L2z8FNkV5v1pqCj55irC7fjxHowPOdcxQdGnAZXvBlZpJOQ_two54my7nmM3lw-dHGp2ZLViQtIHY3FWKfTBpnhefLH3C-IHEgPb9w'

In [125]:
table = dynamodb.Table('SPOTIFY_ZODIAC')
table = tracks_table

def print_table_items_exact(table):
    response = table.scan()
    data = response['Items']

    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        data.extend(response['Items'])

    print(f"there are {len(data)} entries")

# updates every 6 hours
def print_table_items_estimate(table):
    print(f"there are {table.item_count} entries")
    print(f"estimated table storage size: {round(table.table_size_bytes / pow(1024 , 3), 5)} gb")

print_table_items_estimate(table)

#print_table_items_exact(table)
# last checked value was 70784, 1/2/23 10:11pm (39.3 secs run time)

there are 70784 entries
estimated table storage size: 0.02626 gb


download database as csv

In [3]:
def Decimal(number):
    if float(number)%1==0:
        return int(number)
    return float(number)

table = dynamodb.Table('SPOTIFY_TRACKS')

response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

print(f"there are {len(data)} entries")


there are 178907 entries


In [4]:
import csv
import datetime
now = datetime.datetime.now()

with open(f"data/song_database_{now.strftime('%Y-%m-%d_%H-%M')}.csv", 'w', newline='') as csv_file:  
    writer = csv.writer(csv_file)

    # write headers
    writer.writerow(data[0].keys())
    
    # write all values
    for item in data:
        values = list(item.values())
        for i in range(len(values)):
            value = values[i]
            if type(value) == str and value.count('.') == 1:
                value = float(value)
        
        writer.writerow(values)


In [103]:
# Helper functions:

def get_playlist_uri(playlist_url: str):
    return re.search(r'/([^/]+)\?', playlist_url).group(1)

def get_track_ids_and_store_audio_features(access_token: str, playlist_uri: str):
    playlist_info = {
        'track_ids': [],
        'playlist_uri': playlist_uri,
    }

    tracks_table = dynamodb.Table('SPOTIFY_TRACKS')

    headers = {
        'Authorization': f"Bearer {access_token}"
    }

    print(f"fetching playlist {playlist_uri}")
    raw_json_to_check = {'next': f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?market=US&limit=50"}

    num = 0
    while raw_json_to_check['next'] != None:
        print(num)
        response = requests.get(url = raw_json_to_check['next'], headers = headers)

        if response.status_code == 200:
            playlist_raw_json = response.json()

            map_track_ids = []
            for item in playlist_raw_json['items']:
                if item['track'] != None and item['track']['id'] not in playlist_info['track_ids']:
                    track_id = item['track']['id']

                    map_track_ids.append({'id': track_id})
                    playlist_info['track_ids'].append(track_id)
            
            track_ids = []
            index = 0
            while index < len(map_track_ids) - 1:
                db_response = dynamodb.batch_get_item(
                    RequestItems={
                        'SPOTIFY_TRACKS': {
                            'Keys': map_track_ids[index:index + 100]
                        }
                    }
                )
                track_received = db_response['Responses']['SPOTIFY_TRACKS']
                track_ids_received = {}
                for track in track_received:
                    track_ids_received[track['id']] = 1

                # add track if havent been added
                max_value = index + 100
                if max_value > len(map_track_ids):
                    max_value = len(map_track_ids)
                
                for i in range(index, max_value):
                    track_id = map_track_ids[i]['id']

                    if track_id not in track_ids_received:
                        index += 1
                        track_ids.append(track_id)
                
                index += len(track_received) - 1

            if len(track_ids) != 0:
                query = {
                    'ids': ','.join(track_ids)
                }
                response = requests.get(url = 'https://api.spotify.com/v1/audio-features', headers = headers, params = query)

                if response.status_code == 200:
                    raw_json = response.json()
                    
                    i = 0
                    while i < len(raw_json['audio_features']):
                        audio_features = raw_json['audio_features'][i]

                        if audio_features != "None" and audio_features != None:
                            for key, value in audio_features.items():
                                if isinstance(value, float):
                                    raw_json['audio_features'][i][key] = str(value)
                                    
                        else:
                            raw_json['audio_features'].pop(i)
                            i -= 1

                        i += 1
                    
                    for track in raw_json['audio_features']:
                        tracks_table.put_item(Item=track)

                else:
                    print(response.text)

            num += 1
            raw_json_to_check = playlist_raw_json

        else:
            print(response.text)
    
    return playlist_info

# General Spotify Info

In [17]:
categories = []

headers = {
        'Authorization': f"Bearer {access_token}"
}

next_url = 'https://api.spotify.com/v1/browse/categories?country=US&limit=50'

while next_url != None:
    response = requests.get(url = next_url, headers = headers)

    if response.status_code == 200:
        raw_json = response.json()

        for item in raw_json['categories']['items']:
            categories.append({'id': item['id'], 'name': item['name']})

        next_url = raw_json['categories']['next']

    else:
        print(response.text)


for category in categories:
    print(f"{category['name']} - {category['id']}")

Top Lists - toplists
Hip-Hop - 0JQ5DAqbMKFQ00XGBls6ym
Pop - 0JQ5DAqbMKFEC4WFtoNRpw
Country - 0JQ5DAqbMKFKLfwjuJMoNC
Latin - 0JQ5DAqbMKFxXaXKP7zcDp
Rock - 0JQ5DAqbMKFDXXwE9BDJAr
Summer - 0JQ5DAqbMKFLVaM30PMBm4
Workout - 0JQ5DAqbMKFAXlCG6QvYQ4
R&B - 0JQ5DAqbMKFEZPnFQSFB1T
Dance/Electronic - 0JQ5DAqbMKFHOzuVTgTizF
Netflix - 0JQ5DAqbMKFEOEBCABAxo9
Indie - 0JQ5DAqbMKFCWjUTdzaG0e
Mood - 0JQ5DAqbMKFzHmL4tf05da
Sleep - 0JQ5DAqbMKFCuoRTxhYWow
Christian & Gospel - 0JQ5DAqbMKFy0OenPG51Av
Regional Mexican - 0JQ5DAqbMKFDTEtSaS4R92
Wellness - 0JQ5DAqbMKFLb2EqgLtpjC
Chill - 0JQ5DAqbMKFFzDl7qN9Apr
EQUAL - 0JQ5DAqbMKFPw634sFwguI
Gaming - 0JQ5DAqbMKFCfObibaOZbv
Frequency - 0JQ5DAqbMKFF9bY76LXmfI
Kids & Family - 0JQ5DAqbMKFFoimhOqWzLB
Party - 0JQ5DAqbMKFA6SOHvT3gck
Decades - 0JQ5DAqbMKFIVNxQgRNSg0
Fresh Finds - 0JQ5DAqbMKFImHYGo3eTSg
Jazz - 0JQ5DAqbMKFAJ5xb0fwo9m
Focus - 0JQ5DAqbMKFCbimwdOYlsl
Romance - 0JQ5DAqbMKFAUsdyVjCQuL
Folk & Acoustic - 0JQ5DAqbMKFy78wprEpAjl
K-Pop - 0JQ5DAqbMKFGvOw3O4nLAf
Instrum

In [24]:
category_id = '0JQ5DAqbMKFPrEiAOxgac3'


playlist_ids = []

headers = {
        'Authorization': f"Bearer {access_token}"
}

next_url = f"https://api.spotify.com/v1/browse/categories/{category_id}/playlists?limit=50"

while next_url != None:
    response = requests.get(url = next_url, headers = headers)

    if response.status_code == 200:
        raw_json = response.json()

        for item in raw_json['playlists']['items']:
            if item != None:
                playlist_ids.append(item['id'])

        next_url = raw_json['playlists']['next']

    else:
        print(response.text)

print(f"there are {len(playlist_ids)} playlists matching this category")
print(playlist_ids)

there are 108 playlists matching this category
['37i9dQZF1DWWEJlAGA9gs0', '37i9dQZF1DWV0gynK7G6pD', '37i9dQZF1DX4P0ijJK5lUv', '37i9dQZF1DWVFeEut75IAL', '37i9dQZF1DXbPHTEEyQ6Hv', '37i9dQZF1DWUPafHP1BJw1', '37i9dQZF1DX17GkScaAekA', '37i9dQZF1DX4sWSpwq3LiO', '37i9dQZF1DWSPX59wKzEbg', '37i9dQZF1DWTi57LiHYTmH', '37i9dQZF1DWUqIzZNMSCv3', '37i9dQZF1DX5Lm1ZiObdc3', '37i9dQZF1DX2XWJkYVfE4v', '37i9dQZF1DX2XFkp2ZtYHj', '37i9dQZF1DX561TxkFttR4', '37i9dQZF1DX7Ope5RE4ePQ', '37i9dQZF1DWXjj6kdiviS0', '37i9dQZF1DX1jDTenPbqLo', '37i9dQZF1DX2aCk0vzzaZQ', '37i9dQZF1DWXtBjoO4Thyy', '37i9dQZF1DWW7gj0FcGEx6', '37i9dQZF1DX0RmYiHQWCdR', '37i9dQZF1DWXi6GqUgtvam', '37i9dQZF1DX4s3V2rTswzO', '37i9dQZF1DX6DLB6M8zkNk', '37i9dQZF1DX2aCk0vzzaZQ', '37i9dQZF1DWZ27zYbfKITD', '37i9dQZF1DWSZXCWJ6tnaz', '37i9dQZF1DWYRcvdDwEl3O', '37i9dQZF1DWUWUfWSLE7dn', '37i9dQZF1DX9WxEZbyU6MA', '37i9dQZF1DX0CM2xT3aAkN', '37i9dQZF1DWXjj6kdiviS0', '37i9dQZF1DX5hL1aT2vhMb', '37i9dQZF1DX4QrK4j7ie9B', '37i9dQZF1DWY3bSz5dBEak', '37i9dQZF1DX1ACG